# Automated ML

Import dependencies

In [1]:
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Model, Webservice
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

import data_prep

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'ddos_automl_experiment'

experiment=Experiment(ws, experiment_name)

dataset = data_prep.get_DDoS_dataset(ws, key="DDoS Dataset")

## Find or Create Compute Cluster

In [3]:
cpu_cluster_name = "cpu-cluster"

# Check if the cluster exists. If there is an error, create the cluster
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
automl_settings =  {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "featurization": 'auto'
}

automl_config = AutoMLConfig(
                    task="classification",
                    training_data=dataset,
                    label_column_name="Label",
                    compute_target=cpu_cluster,
                    **automl_settings)

In [5]:
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_automl_run, best_automl_model = automl_run.get_output()
best_run_metrics = best_automl_run.get_metrics()

print('Best Run Id: ', best_automl_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(best_automl_model._final_estimator)
print(best_automl_run.get_tags())

Best Run Id:  AutoML_d5dc6914-09aa-4233-a9ad-8766c76c3119_23

 Accuracy: 0.9999843681610705
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('1',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
         

In [ ]:
model = best_automl_run.register_model(model_name = 'best_automl_model', model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [ ]:
rest_service = Model.deploy(ws, "best-model-service", [model])

rest_service.wait_for_deployment(show_output=True)
rest_service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
data = {"data":
        [
          {
            
          },
          {
            
          },
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

In [ ]:
import requests


response = requests.post(rest_service, 
                         headers=auth_header, 
                         json=input_data
                        )

print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = rest_service.get_logs()

for line in logs.split('\n'):
    print(line)